In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns
import pydicom
import time
import gc
import operator 
from apex import amp 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
from apex import amp
import helper
import torchvision.models as models
import pretrainedmodels
from torch.optim import Adam
from functools import partial
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


## Set parameters below

In [11]:
# here you should set which model parameters you want to choose (see definitions.py) and what GPU to use
params=parameters['se_resnet101_5'] # se_resnet101_5, se_resnext101_32x4d_3, se_resnext101_32x4d_5

device=device_by_name("Tesla") # RTX , cpu
torch.cuda.set_device(device)
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'{} results'.format(params['model_name']))

In [12]:
params

{'model_name': 'se_resnet101',
 'SEED': 432,
 'n_splits': 5,
 'Pre_version': None,
 'focal': False,
 'version': 'new_splits',
 'train_prediction': 'predictions_train_tta',
 'train_features': 'features_train_tta',
 'test_prediction': 'predictions_test',
 'test_features': 'features_test',
 'num_epochs': 5,
 'num_pool': 8}

In [13]:
SEED = params['SEED']
n_splits=params['n_splits']

In [14]:
train_df = pd.read_csv(data_dir+'train.csv')
train_df.shape
train_df=train_df[~train_df.PatientID.isin(bad_images)].reset_index(drop=True)
train_df=train_df.drop_duplicates().reset_index(drop=True)
train_df.shape
train_df.head()

(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


In [15]:
test_df = pd.read_csv(data_dir+'test.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


In [16]:
split_sid = train_df.PID.unique()
splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(split_sid))


In [17]:
pickle_file=open(outputs_dir+"PID_splits_{}.pkl".format(n_splits),'wb')
pickle.dump((split_sid,splits),pickle_file,protocol=4)
pickle_file.close()


In [10]:
def my_loss(y_pred,y_true,weights):
    if len(y_pred.shape)==len(y_true.shape): 
        # Normal loss
        loss = F.binary_cross_entropy_with_logits(y_pred,y_true,weights.expand_as(y_pred))
    else:
        # Mixup loss (not used here)
        loss0 = F.binary_cross_entropy_with_logits(y_pred,y_true[...,0],weights.repeat(y_pred.shape[0],1),reduction='none')
        loss1 = F.binary_cross_entropy_with_logits(y_pred,y_true[...,1],weights.repeat(y_pred.shape[0],1),reduction='none')
        loss = (y_true[...,2]*loss0+(1.0-y_true[...,2])*loss1).mean() 
    return loss

In [11]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, y_pred,y_true,weights):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(y_pred,y_true,weights.expand_as(y_pred), reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(y_pred,y_true,weights.expand_as(y_pred), reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [12]:
class parameter_scheduler():
    def __init__(self,model,do_first=['classifier'],num_epoch=1):
        self.model=model
        self.do_first = do_first
        self.num_epoch=num_epoch
    def __call__(self,epoch):
        if epoch>=self.num_epoch:
            for n,p in self.model.named_parameters():
                p.requires_grad=True
        else:
            for n,p in self.model.named_parameters():
                p.requires_grad= any(nd in n for nd in self.do_first)


In [13]:
def get_model(model_name):
    if params['model_name'].startswith('se'):
        return MySENet, pretrainedmodels.__dict__[params['model_name']](num_classes=1000, pretrained='imagenet')
    elif 'Densenet161' in params['model_name']:
        return partial(MyDenseNet, strategy='none'),models.densenet161(pretrained=True)
    elif 'Densenet169' in params['model_name']:
        return partial(MyDenseNet, strategy='none'),models.densenet169(pretrained=True)
    else:
        raise

In [ ]:
%matplotlib nbagg
for num_split in range(params['n_splits']):
    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    #torch.backends.cudnn.deterministic = True
    idx_train = train_df[train_df.PID.isin(set(split_sid[splits[num_split][0]]))].index.values
    idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].index.values
    idx_train.shape
    idx_validate.shape

    klr=1
    batch_size=32
    num_workers=12
    num_epochs=params['num_epochs']
    model_name,version = params['model_name'] , params['version']
    new_model,base_model=get_model(params['model_name'])
    model =  new_model(base_model,
                       len(hemorrhage_types),
                       num_channels=3,
                       dropout=0.2,
                       wso=((40,80),(80,200),(40,400)),
                       dont_do_grad=[],
                       extra_pool=params['num_pool'],
                       )
    if params['Pre_version'] is not None:
        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,params['Pre_version'],
                                                                         num_split),map_location=torch.device(device)))

    _=model.to(device)
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    loss_func=my_loss if not params['focal'] else FocalLoss()
    targets_dataset=D.TensorDataset(torch.tensor(train_df[hemorrhage_types].values,dtype=torch.float))
    transform=MyTransform(mean_change=15,
                          std_change=0,
                          flip=True,
                          zoom=(0.2,0.2),
                          rotate=30,
                          out_size=512,
                          shift=10,
                          normal=False)
    imagedataset = ImageDataset(train_df,transform=transform.random,base_path=train_images_dir,
                               window_eq=False,equalize=False,rescale=True)
    transform_val=MyTransform(out_size=512)
    imagedataset_val = ImageDataset(train_df,transform=transform_val.random,base_path=train_images_dir,
                                   window_eq=False,equalize=False,rescale=True)
    combined_dataset=DatasetCat([imagedataset,targets_dataset])
    combined_dataset_val=DatasetCat([imagedataset_val,targets_dataset])
    optimizer_grouped_parameters=model.get_optimizer_parameters(klr)
    sampling=sampler(train_df[hemorrhage_types].values[idx_train],0.5,[0,0,0,0,0,1])
    sample_ratio=1.02*float(sampling().shape[0])/idx_train.shape[0]
    train_dataset=D.Subset(combined_dataset,idx_train)
    validate_dataset=D.Subset(combined_dataset_val,idx_validate)
    num_train_optimization_steps = num_epochs*(sample_ratio*len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    fig,ax = plt.subplots(figsize=(10,7))
    gr=loss_graph(fig,ax,num_epochs,int(num_train_optimization_steps/num_epochs)+1,limits=(0.05,0.2))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=num_epochs,tau=1)
    optimizer = BertAdam(optimizer_grouped_parameters,lr=klr*1e-3,schedule=sched)
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    model_apexed=True,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=None,
                                    return_model=True,
                                    num_workers=num_workers,
                                    sampler=None,
                                    pre_process = None,
                                    graph=gr,
                                    call_progress=sendmeemail)

    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

In [ ]:
for num_split in range(params['n_splits']):
    idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].index.values
    model_name,version =params['model_name'] , params['version']
    new_model,base_model=get_model(params['model_name'])
    model =  new_model(base_model,
                       len(hemorrhage_types),
                       num_channels=3,
                       dropout=0.2,
                       wso=((40,80),(80,200),(40,400)),
                       dont_do_grad=[],
                       extra_pool=params['num_pool'],
                       )
    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    _=model.to(device)
    transform=MyTransform(mean_change=15,
                          std_change=0,
                          flip=True,
                          zoom=(0.2,0.2),
                          rotate=30,
                          out_size=512,
                          shift=0,
                          normal=False)
    indexes=np.arange(train_df.shape[0]).repeat(4)
    train_dataset=D.Subset(ImageDataset(train_df,transform=transform.random,base_path=train_images_dir,
                              window_eq=False,equalize=False,rescale=True),indexes)
    pred,features = model_run(model,train_dataset,do_apex=True,batch_size=96,num_workers=14)

    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,params['train_features'],num_split),'wb')
    pickle.dump(features,pickle_file,protocol=4)
    pickle_file.close()

    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,params['train_prediction'],num_split),'wb')
    pickle.dump(pred,pickle_file,protocol=4)
    pickle_file.close()


    my_loss(pred[(idx_validate*4+np.arange(4)[:,None]).transpose(1,0)].mean(1),
            torch.tensor(train_df[hemorrhage_types].values[idx_validate],dtype=torch.float),
            torch.tensor([1.,1.,1.,1.,1.,2.]))

In [ ]:
for num_split in range(params['n_splits']):
    idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].index.values
    model_name,version =params['model_name'] , params['version']
    new_model,base_model=get_model(params['model_name'])
    model =  new_model(base_model,
                       len(hemorrhage_types),
                       num_channels=3,
                       dropout=0.2,
                       wso=((40,80),(80,200),(40,400)),
                       dont_do_grad=[],
                       extra_pool=params['num_pool'],
                       )
    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    _=model.to(device)
    transform=MyTransform(mean_change=15,
                          std_change=0,
                          flip=True,
                          zoom=(0.2,0.2),
                          rotate=30,
                          out_size=512,
                          shift=0,
                          normal=False)
    indexes=np.arange(test_df.shape[0]).repeat(8)
    imagedataset_test=D.Subset(ImageDataset(test_df,transform=transform.random,base_path=test_images_dir,
                                  window_eq=False,equalize=False,rescale=True),indexes)
    pred,features = model_run(model,imagedataset_test,do_apex=True,batch_size=96,num_workers=18)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,params['test_features'],num_split),'wb')
    pickle.dump(features,pickle_file,protocol=4)
    pickle_file.close()

    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,params['test_prediction'],num_split),'wb')
    pickle.dump(pred,pickle_file,protocol=4)
    pickle_file.close()


## create submission file - for reference

In [33]:
preds=[]
for i in tqdm_notebook(range(params['n_splits'])):
    model_name,version, num_split =  params['model_name'] , params['version'],i
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,params['test_prediction'],num_split),'rb')
    pred=pickle.load(pickle_file)
    pickle_file.close()
    preds.append(pred[(np.arange(pred.shape[0]).reshape(pred.shape[0]//8,8))])
predss = torch.cat(preds,1)
predss.shape

torch.Size([78545, 24, 6])

In [36]:
submission_df=get_submission(test_df,torch.sigmoid(predss).mean(1),False)
submission_df.head(12)
submission_df.shape
sub_num=999
submission_df.to_csv('/media/hd/notebooks/data/RSNA/submissions/submission{}.csv'.format(sub_num),
                                                                  index=False, columns=['ID','Label'])


,ID,Label
0,ID_000012eaf_any,0.012404
1,ID_000012eaf_epidural,0.000464
2,ID_000012eaf_intraparenchymal,0.001818
3,ID_000012eaf_intraventricular,0.000576
4,ID_000012eaf_subarachnoid,0.001655
5,ID_000012eaf_subdural,0.010707
6,ID_0000ca2f6_any,0.002507
7,ID_0000ca2f6_epidural,0.000038
8,ID_0000ca2f6_intraparenchymal,0.000540
9,ID_0000ca2f6_intraventricular,0.000080


(471270, 2)